In [1]:
import pandas as pd
import numpy as np

#from sklearn.impute import SimpleImputer
#from sklearn.preprocessing import StandardScaler, OneHotEncoder
#from sklearn.compose import make_column_transformer



#from sklearn.pipeline import make_pipeline
#from sklearn.pipeline import Pipeline



from preprocessing_functions import *
from problem_functions import *

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
production = pd.read_sas('C:/Users/kucza/OneDrive/Desktop/DataScience/CreditScoringPython/dane/production.sas7bdat',
                        format = 'sas7bdat',
                         encoding="ISO-8859-1" )
transactions = pd.read_sas('C:/Users/kucza/OneDrive/Desktop/DataScience/CreditScoringPython/dane/transactions.sas7bdat',
                        format = 'sas7bdat',
                         encoding="ISO-8859-1" )

In [ ]:
production.head()

In [ ]:
transactions.head()

Default definition: if a product ever has n_installments_due or more it is considered as default, no matter if the client pays the loan back at a later time.

In [ ]:
#transformer_in_columns = transformer.transformers_[:-1]
#raw_col_name = transformer_in_columns[1]
#raw_col_name[1].get_feature_names()

In [4]:
data = MaxDueInstallmentsMinPeriod(transactions)
def_product = DefaultPerProduct(data, 3, 200712)
data_merged = ProductionDefault(def_product, production)
data_merged_nomisscol = DeleteMissing(data_merged, 0.6)

data_css = data_merged_nomisscol[data_merged_nomisscol['product'] == 'css']
data_css_select = data_css.iloc[:,5:].copy()
y = data_css.loc[:,'default3'].values
data_css_select.drop('default3', axis = 1, inplace = True)

In [6]:
display(unique_count(data_css_select))

,index,Column_Name,Num_Unique
0,22,act_cus_active,1
1,3,app_loan_amount,1
2,4,app_n_installments,1
3,74,APP_CHAR_BRANCH,1
4,7,app_installment,1
5,76,APP_CHAR_CARS,2
6,77,APP_CHAR_HOME_STATUS,2
7,60,ags12_Min_CMaxA_Due,2
8,72,APP_CHAR_GENDER,2
9,51,ags9_Min_CMaxA_Due,3


In [5]:
#drop_cols = unique_count(data_css_select)['Column_Name'][0:5].values.tolist()
cat_cols = unique_count(data_css_select)['Column_Name'][5:57].values.tolist()
num_cols = unique_count(data_css_select)['Column_Name'][57:].values.tolist()

data_X = Imputer_df(dataframe = data_css_select, num_cols_names = num_cols, cat_cols_names = cat_cols)
data_X_onehot = MakeDummiesAndScale(data_X, cat_cols, num_cols)
X_train, X_test, y_train, y_test = train_test_split_df(data_X_onehot, y)

In [11]:
X_train.to_csv('X_train.csv', index = False)
X_test.to_csv('X_test.csv', index = False)
y_train.to_csv('y_train.csv', index = False)
y_test.to_csv('y_test.csv', index = False)

## Gradient Boosting

In [ ]:
lin_reg = LogisticRegression()
lin_reg.fit(X_train, y_train)

In [ ]:
y_pred_prob = lin_reg.predict_proba(X_test)
y_pred = lin_reg.predict(X_test)

In [ ]:
y_pred_prob_plot = [y_pred_prob[i][1] for i in range(len(y_pred_prob))]
plt.hist(y_pred_prob_plot, bins=100)
plt.show()

In [ ]:
cv = cross_val_score(lin_reg, data_X, y, cv = 5, scoring = 'f1')

In [ ]:
print("avg. accuracy = "+str(round(np.mean(cv),4))+
      " std. of accuracy = "+str(round(np.std(cv),4))+
      " std. coef = "+str(round(np.std(cv)/np.mean(cv),4)))

In [ ]:
print(cm(y_true = y_test, y_pred = y_pred))

In [ ]:
get_column_names_from_ColumnTransformer(transformer)

In [ ]:
data_merged_nomisscol.head()

In [ ]:
data_merged_nomiss.isna().mean().sort_values(ascending=False)

In [ ]:
plt.hist(data_merged_nomiss.isna().mean().sort_values(ascending=False), bins=100)
plt.show()

In [ ]:
plt.hist(x = unique_count(data_merged_nomiss)['Num_Unique'], bins = 10000)
plt.xlim(0, 100)
plt.show()

In [ ]:
len(data_merged)